In [28]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [18]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
STUDENT_ROLLNO = "IMT2020548" #yourRollNumberHere
STUDENT_NAME = "Tejas Sharma" #yourNameHere
#@PROTECTED_1
##DO NOT MODIFY THE BELOW CODE. NO OTHER IMPORTS ALLOWED. NO OTHER FILE LOADING OR SAVING ALLOWED.
import torch.nn as nn 
import torch.optim as optim 
import torchmetrics
import torch
import sklearn.preprocessing as preprocessing
import sklearn.model_selection as model_selection 
import torch.utils.data as data 
import numpy as np 
from tqdm import tqdm
X_train = np.load("/content/drive/MyDrive/ML_Ass2/Task2/task2/X_train.npy")
y_train = np.load("/content/drive/MyDrive/ML_Ass2/Task2/task2/y_train.npy")
X_test = np.load("/content/drive/MyDrive/ML_Ass2/Task2/task2/X_test.npy")
submission = np.load("/content/drive/MyDrive/ML_Ass2/Task2/task2/sample_submission.npy")
#@PROTECTED_1

#Protected_2 has been moved to the end unedited

In [29]:
torch.cuda.is_available()

False

In [21]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [22]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-kt80aj7m
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-kt80aj7m


In [24]:
Scaler = preprocessing.StandardScaler()

In [25]:
X_train = Scaler.fit_transform(X_train)
X_test = Scaler.fit_transform(X_test)

In [ ]:
labels = np.zeros((y_train.size, y_train.max()+1))
labels[np.arange(y_train.size),y_train] = 1

In [26]:
trainset_X, testset_X, trainset_y, testset_y = model_selection.train_test_split(X_train, y_train, test_size=0.3)

In [ ]:
# X_tr = X_train[:40000]
# Y_train = labels[:40000]
# X_test = X_train[40001:]
# Y_labels = labels[40001:]
# X_test

In [ ]:
# Y_train = torch.from_numpy(Y_train).cuda().float()
# X_train = torch.from_numpy(X_tr).cuda().float()
# X_test = torch.from_numpy(X_test).cuda().float()
# Y_test = torch.from_numpy(Y_labels).cuda().float()

In [27]:
X_test = torch.from_numpy(testset_X).cuda().float()
Y_test = torch.from_numpy(testset_y).cuda().float()
X_train = torch.from_numpy(trainset_X).cuda().float()
Y_train = torch.from_numpy(trainset_y).cuda().float()

RuntimeError: ignored

In [ ]:
dataset = data.TensorDataset(X_train, y_train)
dataset

In [ ]:
model = nn.Sequential(
      nn.Linear(3072,512),
      nn.ELU(alpha=1.5),
      nn.Dropout(0.25),

      # nn.Linear(1024,512),
      # nn.ELU(alpha=1.5),
      # nn.Dropout(0.25),

      nn.Linear(512,256),
      nn.ELU(alpha=1.5),
      nn.Dropout(0.25),

      nn.Linear(256,128),
      nn.ELU(alpha=1.5),
      nn.Dropout(0.25),

      nn.Linear(128,64),
      nn.ELU(alpha=1.5),
      nn.Dropout(0.25),
      nn.Linear(64, 10),
    ).cuda()

In [ ]:
opt = optim.Adam(model.parameters(), lr = 0.001)
Loss = nn.CrossEntropyLoss().cuda()

epochs = 400
costs = []
accuracies = []
test_costs = []
test_accuracies = []

In [ ]:
def train_model(epochs, dataset, model):
  opt = optim.Adam(model.parameters(), lr = 0.001)
  Loss = nn.CrossEntropyLoss().cuda()

  losses = []

  for epoch in tqdm(range(epochs)):
      opt.zero_grad()
      preds = model(X)
      Entropy_loss = Loss(preds,Y.long())
      Entropy_loss.backward()
      opt.step()
      with torch.no_grad(): 
        costs.append(Entropy_loss.detach().cpu().numpy().item()) 
        accuracies.append(acc(preds, y_train).cpu().numpy().item())
        testset_yhat_tens = model(X_test) 
        test_costs.append(Loss(testset_yhat_tens, Y_test).cpu().numpy().item()) 
        test_accuracies.append(acc(testset_yhat_tens, Y_test).cpu().numpy().item())
      train_dl = data.DataLoader(dataset, batch_size = 5000, shuffle = True)
      for _,batch in enumerate(train_dl,0):
          (X,Y) = (batch[0],batch[1])
          losses.append(Entropy_loss.item())
  return np.array(losses)

In [ ]:
losses = train_model(epochs, dataset, model)

In [ ]:
loss_data_avgd = losses.reshape(500, -1).mean(axis = 1)
loss_data_avgd

In [ ]:
model.eval()

In [ ]:
y_pred = model(X_test)
y_pred = torch.argmax(y_pred, axis=1)

In [ ]:
y_pred

In [ ]:
Y_test = np.load("/content/y_test.npy")

In [ ]:
Y_test = torch.from_numpy(Y_test).cuda()

In [ ]:
Y_test

In [ ]:
acc = torchmetrics.Accuracy(task="multiclass",num_classes=10).to('cuda')
acc(Y_test, y_pred).cpu().numpy().item()

In [ ]:
#@PROTECTED_2
# np.save("{}__{}".format(STUDENT_ROLLNO,STUDENT_NAME))
#@PROTECTED_2